<a href="https://colab.research.google.com/github/kulbir-ahluwalia/CMSC818B_MiniProject_2/blob/main/python/pm_python.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
## Get the required python files
!wget https://raw.githubusercontent.com/kulbir-ahluwalia/CMSC818B_MiniProject_2/main/python/actors.py
!wget https://raw.githubusercontent.com/kulbir-ahluwalia/CMSC818B_MiniProject_2/main/python/env.py

--2020-12-02 04:46:20--  https://raw.githubusercontent.com/kulbir-ahluwalia/CMSC818B_MiniProject_2/main/python/actors.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5622 (5.5K) [text/plain]
Saving to: ‘actors.py.2’

actors.py.2         100%[===================>]   5.49K  --.-KB/s    in 0s      

2020-12-02 04:46:20 (58.7 MB/s) - ‘actors.py.2’ saved [5622/5622]

--2020-12-02 04:46:20--  https://raw.githubusercontent.com/kulbir-ahluwalia/CMSC818B_MiniProject_2/main/python/env.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 6874 (6.7K) 

In [2]:
!pip install pygame

In [3]:
%load_ext autoreload
%autoreload 2

In [4]:
import cv2
import numpy as np
from google.colab.patches import cv2_imshow
from google.colab import output
import time 
import os, sys
import pygame


# set SDL to use the dummy NULL video driver, 
#   so it doesn't need a windowing system.
os.environ["SDL_VIDEODRIVER"] = "dummy"

pygame 2.0.0 (SDL 2.0.12, python 3.6.9)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [5]:
## Import classes for elements and visualization
from actors import Player, Obstacle, Drone
from env import Canvas, process_into_image, process_img, get_obstacles, get_droneview, update_all, process_screen

### Defining the environement

In [13]:
## Canvas is the grid we are going to use
canvas = Canvas(300,300)

## Create the robots
playerList = []
playerList.append(Player(pos=[20, 10], color='g', size=10))
playerList.append(Player(pos=[0, 260], color='g', size=10))
playerList.append(Player(pos=[200, 10], color='g', size=10))
playerList.append(Player(pos=[250, 200], color='g', size=10))
# playerList.append(Player(pos=[200, 300], color='g', size=10))
# playerList.append(Player(pos=[400, 400], color='g', size=10))

## Create teh robots at random locations
n_obj = 20 # number of objects
obstacleList = []
# obstacleList.append(Obstacle(pos=[60, 60], size=[10,15]))
# obstacleList.append(Obstacle(pos=[60, 60],size=[20,10]))
r_coords = np.random.randint(0, canvas.height, (n_obj)) # random rows
c_coords = np.random.randint(0, canvas.width, (n_obj)) # random columns
# Width and height would be chosen from 10,15,20,25,30 randomly
for i in range(len(r_coords)):
    obstacleList.append(Obstacle(pos=[r_coords[i], c_coords[i]], 
                                 size=[np.random.choice([10,15,20,25,30]),np.random.choice([10,15,20,25,30])]))


### Add drone to the environememnt
droneList = []
droneList.append(Drone(pos=[150,150], color='b',  size=100))

### Algorithms for testing

In [7]:
# pygame.init()

## Step size. How many grids would the robot move at each step
step_size = playerList[0].size

## the dictionary defniing how many to update the location
move_dict = {
    'left' : np.array([0, -step_size]),
    'up' : np.array([-step_size,0]),
    'right' : np.array([0, step_size]),
    'down' : np.array([step_size,0])
}


    


#### Lawnmower algorithm with *hover over buildings*

In [8]:
## Order of moves for anti-clockwise movement
anti_clock_order_dict = {'up' : 0,
                    'right': 1,
                    'down' : 2,
                    'left': 3}
anti_clock_order = ['down', 'right', 'up', 'left']

## Function to check if a location is valid (within grid)
def is_valid(pos, limits):
    if ((pos[0] < 0) or (pos[1] < 0) or (pos[0] >= limits[0]) or (pos[1] >= limits[1])):
        return False
    else:
        return True

## The lawnmower algorithm
def lawn_mover(player, data_img):
    pos = player.pos  # Get players location
    obstacle = (data_img[:,:,0] > 0).astype(int) # get grid indicating where objects are
    covered = (data_img[:,:,1] > 0).astype(int) # get grid indicating which areas have been covered
    # covered[data_img[:,:,1] == 255] = 0

    # if the last action is not recorded, move to left or right as per the row
    if player.prev_action == None:
        player.prev_action = ['right', 'left'][pos[1]%2]
    

    ####
    ## Iterate over each direction, till yu find a valid orientation to move
    for i in range(4):
        player.prev_action = anti_clock_order[(anti_clock_order_dict[player.prev_action]+i) %4]
        move = move_dict[player.prev_action]
        print('CHECK: ', pos, move, player.prev_action)
        temp_pos = pos + move
        if not is_valid(temp_pos, data_img.shape[:2]):
            continue
        else:
            if (covered[temp_pos[0], temp_pos[1]] == 1):# or (obstacle[temp_pos[0], temp_pos[1]] == 1):
            # if (obstacle[temp_pos[0], temp_pos[1]] == 1):
                continue
            else:
                break
    return temp_pos



#### Greedy movement

In [9]:
## Greedy algorithm (distributed)
def greedy_algorithm(player, data_img):
    pos = player.pos #get player location
    obstacle = data_img[:,:,2].astype(int) # get grid indicating where objects are
    covered = (data_img[:,:,1]).astype(int) # get grid indicating which areas have been covered and how much latency is there
    # covered[data_img[:,:,1] == 255] = 0
    cost_grid = obstacle*10 + covered # give high cost to obstacles

    # if the last action is not recorded, move to left or right as per the row
    action_list = ['right', 'down', 'left', 'up']
    if player.prev_action == None:
        player.prev_action = ['right', 'left'][pos[0]%2]


    #######
    ## Looks into future move for all 4 directions, move whereever robot would have least cost
    temp_action_list = []
    temp_cost_list = []
    for i in range(4):
        new_action = action_list[ (action_list.index(player.prev_action) + i) % 4 ]
        move = move_dict[ new_action ]
        
        temp_pos = pos + move
        # print('CHECK: ', pos, temp_pos, new_action)
        if not is_valid(temp_pos, data_img.shape[:2]):
            # print('NOT Valid: ', pos, temp_pos)
            temp_action_list.append(new_action)
            temp_cost_list.append(50000)
            # continue
        else:
            # if (covered[temp_pos[0], temp_pos[1]] == 1) or (obstacle[temp_pos[0], temp_pos[1]] == 1):
            temp_action_list.append(new_action)
            temp_cost = np.sum(cost_grid[max(0,temp_pos[0]-player.size):min(temp_pos[0]+player.size, data_img.shape[0]), 
                                         max(0, temp_pos[1]-player.size): min(temp_pos[1]+player.size,data_img.shape[1])])
            temp_cost_list.append(temp_cost)

                # print('OCCUPIED: ', covered[temp_pos[0], temp_pos[1]], obstacle[temp_pos[0], temp_pos[1]] )
            #     continue
            # else:
            #     break

    # print('Costs: ', temp_cost_list)
    # print('Actions: ', temp_action_list)

    # Choose next action based on where the cost would be least
    new_action = temp_action_list[np.argmin(temp_cost_list)]
    temp_pos = pos + move_dict[ new_action ]

    # update last action
    player.prev_action = new_action
    
    # if teh move is not valid, do not move
    if not is_valid(temp_pos, data_img.shape[:2]):
        temp_pos = pos
        
    return temp_pos


In [14]:
### Envoronemtn image as a numpy array
data_img = np.zeros(canvas.grid.shape+(3,))
# data_img = init_env(canvas, playerList, obstacleList, droneList)
### Coverage information
coverage = np.zeros(canvas.grid.shape, dtype=np.uint8)
#### Drone Map
drone_map = np.zeros(canvas.grid.shape+(3,), dtype=np.uint8)
## Run for 300 iterations
latency_factor = 2
for i in range(300):

    ### Get next location for each robot
    for player in playerList:
        # player.pos = lawn_mover(player, data_img)
        player.pos = greedy_algorithm(player, data_img)
        # player.move(np.random.choice(['up', 'down', 'left', 'right']), player.size, canvas)


    for drone in droneList:
         drone.move(np.random.choice(['up', 'down', 'left', 'right']), drone.size//10, canvas)   

    update_all(canvas, playerList, obstacleList, droneList) 


    covergae, data_img = process_screen(my_coverage=coverage, my_canvas=canvas)
    obstacle_map = get_obstacles(data_img)
    
    # drone_map = get_droneview(drone_map, data_img, latency_factor)
    drone_map = get_droneview(drone_map, data_img, latency_factor, droneList)
    #Display image, clear cell every 0.5 seconds
    #convert image so it can be displayed in OpenCV
    ## UNCOMMENT IN CASE OF ISSUE
    # view = pygame.surfarray.array3d(canvas.screen)
    
    #  convert from (width, height, channel) to (height, width, channel)
    ## UNCOMMENT  NEXT 2 LINES IN CASE OF ERROR
    # view = view.transpose([1, 0, 2])
    # img_bgr = cv2.cvtColor(view, cv2.COLOR_RGB2BGR)

    # cv2_imshow(data_img) #img_bgr[yv, xv])
    print('\t\t Environment \t\t\t Drone View ')
    cv2_imshow(np.hstack([data_img, drone_map]))
    print('\t\t Coverage \t\t\t Obstacles ')
    cv2_imshow(np.hstack([coverage, 255*obstacle_map]))
    

    ## Wait 0.1 second before running next iteration
    time.sleep(0.1)
    output.clear()

    # if i == 3:
    #     break

